In [40]:
import json
from datasets import load_dataset
from tqdm.auto import tqdm

In [42]:
rag_dataset = load_dataset("DuyTa/vi_RAG", 'BKAI_RAG')

In [43]:
rag_dataset1 = load_dataset("DuyTa/vi_RAG", 'LegalRAG')

In [44]:
rag_dataset2 = load_dataset("DuyTa/vi_RAG", 'expert')

In [45]:
instances_list = []

In [46]:
def create_data_json(rag_dataset, group_name):
    # Iterate over each instance in the 'train' split
    for instance in rag_dataset['train']:
        # Create a dictionary for the current instance and append it to the list
        instances_list.append({
            "group": group_name,
            "context": ''.join(instance["context"][0]),
            "question": instance["question"],
            "answer": instance["answer"]
        })

In [47]:
create_data_json(rag_dataset, "General")
create_data_json(rag_dataset1, "Legal")

In [48]:
for instance in rag_dataset2['train']:
        # Create a dictionary for the current instance and append it to the list
        instances_list.append({
            "group": "Expert",
            "context": instance["revised_claims"],
            "question": instance["question"],
            "answer": instance["revised_answer"]
        })

In [49]:
instances_list[0]

{'group': 'General',
 'context': "ngữ cảnh 1: Vượt qua nỗi sợ hãi, Minh Tú xuất sắc vào chung kết Asia's Next Top Model.  Bỏ qua hết những lo lắng về căn bệnh tim, Minh Tú đã hết sức chế ngự sự sợ hãi và hoàn thành tốt phần thử thách đi catwalk khi bị treo lơ lửng trên một tòa nhà cao tầng và dành chiếc vé vào đêm chung kết Asia’s Next Top Model..  Tối ngày 21/6, tập 12 của chương trình Asia’ Next Top Model mùa thứ 5 đã được lên sóng. 4 thí sinh cùng giám khảo bí mật Xiao Qing bước vào phần thi thử thách đặc biệt là đi catwalk khi bị treo lơ lửng trên một tòa nhà cao tầng. Các cô gái phải vừa tạo dáng vừa thể hiện được sự mạnh mẽ, quyến rũ riêng.. Mặc dù có bệnh về tim nhưng Minh Tú đã cố gắng hết sức chế ngự sự sợ hãi và hoàn thành tốt phần thử thách này. Thí sinh chiến thắng là Shikin và đại diện Việt Nam cũng không kém cạnh gì khi xuất sắc xếp ở vị trí thứ 2, báo GĐVN đưa tin.. Bước vào thử thách, Minh Tú hét lớn: “Tôi còn nhiều việc chưa làm được đâu” vì sợ sẽ rơi xuống tuy nhiên c

In [50]:
for doc in tqdm(instances_list):
    context = doc['context']
    if context is None:
        doc['context'] = 'Ngữ cảnh không được cung cấp'
        print("No context")

100%|█████████████████| 6089/6089 [00:00<00:00, 1310168.63it/s]

No context
No context
No context


In [51]:
# Save the list of instances to a JSON file
with open('../data/vietnamese_rag/documents.json', 'w', encoding='utf-8') as f:
    json.dump(instances_list, f, ensure_ascii=False, indent=4)

In [52]:
with open('../data/vietnamese_rag/documents.json', 'rt') as f_in:
    documents = json.load(f_in)

## Create documents with ID


In [53]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['group']}-{doc['question']}-{doc['context']}-{doc['answer']}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [54]:
with open('../data/vietnamese_rag/documents-with-ids.json', 'wt') as f_out:
    json.dump(documents , f_out, indent=2)